In [1]:
# EDA inicial

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [15]:
pip install pyarrow

   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB 393.8 kB/s eta 0:01:05
   ---------------------------------------- 0.3/25.3 MB 2.1 MB/s eta 0:00:12
   - -------------------------------------- 1.0/25.3 MB 5.9 MB/s eta 0:00:05
   --- ------------------------------------ 2.1/25.3 MB 8.9 MB/s eta 0:00:03
   ---- ----------------------------------- 2.9/25.3 MB 10.3 MB/s eta 0:00:03
   ----- ---------------------------------- 3.7/25.3 MB 11.4 MB/s eta 0:00:02
   ------- -------------------------------- 4.8/25.3 MB 13.2 MB/s eta 0:00:02
   --------- ------------------------------ 5.9/25.3 MB 14.5 MB/s eta 0:00:02
   ---------- ----------------------------- 6.8/25.3 MB 15.0 MB/s eta 0:00:02
   ------------ --------------------------- 7.9/25.3 MB 15.3 MB/s eta 0:00:02
   ---------

In [19]:
import os

print(os.listdir("data"))


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'data'

In [18]:
pd.read_parquet("data/reservas_hoteles.parquet", engine="fastparquet")  # o engine="fastparquet"



FileNotFoundError: [Errno 2] No such file or directory: 'data/reservas_hoteles.parquet'